# Don't run this code
I would strongly advise against running this code as it takes several minutes to process all of the data, as well as requiring a few datasets that aren't in shared data. I've included it for documentation purposes and such but I made much more simplified summary datasets from the data that are saved to shared data. 

This code creates four datasets:
1. County_Building_Counts: Number of reported commercial buildings in each county
2. County_Building_Counts_By_Type: Number of reported commercial buildings in each county, further subdivided by building type (Hospitality, Retail, etc.)
3. County_Building_Areas: Total area of buildings in each county
4. County_Building_Areas_Adjusted: Total area of buildings in each county, divided by land area (sq mi) of county (Building density). Probably the best one to use. 

In [1]:
import pandas as pd
import os
import geopandas as gpd

current_dir = os.getcwd()

In [2]:
west_df = pd.read_excel(os.path.join(current_dir, '..', 'Data', 'Buildings', 'West.xlsb'), sheet_name="County")
print("Processed west!")
midwest_df = pd.read_excel(os.path.join(current_dir, '..', 'Data', 'Buildings', 'Midwest.xlsb'), sheet_name="County")
print("Processed midwest!")
southatlantic_df = pd.read_excel(os.path.join(current_dir, '..', 'Data', 'Buildings', 'SouthAtlantic.xlsb'), sheet_name="County")
print("Processed south Atlantic!")
southcentral_df = pd.read_excel(os.path.join(current_dir, '..', 'Data', 'Buildings', 'SouthCentral.xlsb'), sheet_name="County")
print("Processed south central!")
northeast_df = pd.read_excel(os.path.join(current_dir, '..', 'Data', 'Buildings', 'Northeast.xlsb'), sheet_name="County")
print("Processed northeast!")

Processed west!
Processed midwest!
Processed south Atlantic!
Processed south central!
Processed northeast!


In [3]:
df = pd.concat([west_df, midwest_df, southatlantic_df, southcentral_df, northeast_df])
len(df)

2246485

In [4]:
copy_df = df.copy()
copy_df.head()

,Unnamed: 0,city_name,Unnamed: 2,countyname,statefp,statecode,bldg_count,reported_propertytype,reported_propertysubtype,rentablebuildingarea_mean,...,cbecs,Unnamed: 15,area_sum,Unnamed: 17,placeid,countyid,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,NaN,Other Aleutians East Borough,NaN,Aleutians East Borough,2,AK,1,Hospitality,Hotel,5200,...,Hotel,NaN,5200,NaN,NaN,2013,NaN,NaN,NaN,NaN
1,NaN,Other Aleutians East Borough,NaN,Aleutians East Borough,2,AK,1,Hospitality,Motel,5200,...,Motel or inn,NaN,5200,NaN,NaN,2013,NaN,NaN,NaN,NaN
2,NaN,Other Aleutians East Borough,NaN,Aleutians East Borough,2,AK,1,Multi-Family,Apartments,5000,...,Apartments,NaN,5000,NaN,NaN,2013,NaN,NaN,NaN,NaN
3,NaN,Other Aleutians East Borough,NaN,Aleutians East Borough,2,AK,1,Specialty,Lodge/Meeting Hall,16700,...,Other public assembly,NaN,16700,NaN,NaN,2013,NaN,NaN,NaN,NaN
4,NaN,Unalaska,NaN,Aleutians West Census Area,2,AK,1,Multi-Family,Apartments,15200,...,Apartments,NaN,15200,NaN,280770.0,2016,NaN,NaN,NaN,NaN


In [59]:
grouped = copy_df[["statecode", "countyname", "bldg_count"]].groupby(["statecode", "countyname"]).agg("sum").reset_index()
sorted_grouped = grouped.sort_values("bldg_count", ascending=False)
sorted_grouped.head(10)

,statecode,countyname,bldg_count
204,CA,Los Angeles County,175867
709,IL,Cook County,69853
2621,TX,Harris County,51408
222,CA,San Diego County,46896
2003,NY,Kings County,45666
178,AZ,Maricopa County,45030
215,CA,Orange County,42663
362,FL,Miami-Dade County,36276
2577,TX,Dallas County,33395
2010,NY,New York County,32171


In [6]:
sorted_grouped.to_csv(os.path.join(current_dir, '..', 'SharedData', 'County_Building_Counts.csv'), index=False)

In [16]:
grouped_use = copy_df[["statecode", "countyname", "reported_propertytype", "bldg_count", "area_sum"]].groupby(["statecode", "countyname", "reported_propertytype"]).agg("sum").reset_index()
# grouped_stories.sort_values("stories", ascending=False).head(20)
grouped_use.head(10)

,statecode,countyname,reported_propertytype,bldg_count,area_sum
0,AK,Aleutians East Borough,Hospitality,2,10400
1,AK,Aleutians East Borough,Multi-Family,1,5000
2,AK,Aleutians East Borough,Specialty,1,16700
3,AK,Aleutians West Census Area,Multi-Family,1,15200
4,AK,Aleutians West Census Area,Office,2,92300
5,AK,Aleutians West Census Area,Retail,4,59500
6,AK,Anchorage Municipality,Flex,143,2812900
7,AK,Anchorage Municipality,Health Care,11,2115500
8,AK,Anchorage Municipality,Hospitality,89,4868500
9,AK,Anchorage Municipality,Industrial,684,13847400


In [8]:
grouped_area = copy_df[["statecode", "countyname", "area_sum"]].groupby(["statecode", "countyname"]).agg("sum").reset_index()
grouped_area.sort_values("area_sum", ascending=False)

,statecode,countyname,area_sum
204,CA,Los Angeles County,3450593400
709,IL,Cook County,2147458100
2621,TX,Harris County,2001257800
2010,NY,New York County,1822910500
2577,TX,Dallas County,1505328800
...,...,...,...
1849,NE,Hayes County,1400
1545,MS,Issaquena County,1400
377,FL,St Lucie County,1100
2857,VA,Hampton City,1000


In [9]:
def translate_state(state_name):
    state_abbreviations = {
        'Alabama': 'AL',
        'Alaska': 'AK',
        'Arizona': 'AZ',
        'Arkansas': 'AR',
        'California': 'CA',
        'Colorado': 'CO',
        'Connecticut': 'CT',
        'Delaware': 'DE',
        'District of Columbia': 'DC',
        'Florida': 'FL',
        'Georgia': 'GA',
        'Hawaii': 'HI',
        'Idaho': 'ID',
        'Illinois': 'IL',
        'Indiana': 'IN',
        'Iowa': 'IA',
        'Kansas': 'KS',
        'Kentucky': 'KY',
        'Louisiana': 'LA',
        'Maine': 'ME',
        'Maryland': 'MD',
        'Massachusetts': 'MA',
        'Michigan': 'MI',
        'Minnesota': 'MN',
        'Mississippi': 'MS',
        'Missouri': 'MO',
        'Montana': 'MT',
        'Nebraska': 'NE',
        'Nevada': 'NV',
        'New Hampshire': 'NH',
        'New Jersey': 'NJ',
        'New Mexico': 'NM',
        'New York': 'NY',
        'North Carolina': 'NC',
        'North Dakota': 'ND',
        'Ohio': 'OH',
        'Oklahoma': 'OK',
        'Oregon': 'OR',
        'Pennsylvania': 'PA',
        'Rhode Island': 'RI',
        'South Carolina': 'SC',
        'South Dakota': 'SD',
        'Tennessee': 'TN',
        'Texas': 'TX',
        'Utah': 'UT',
        'Vermont': 'VT',
        'Virginia': 'VA',
        'Washington': 'WA',
        'West Virginia': 'WV',
        'Wisconsin': 'WI',
        'Wyoming': 'WY'
    }

    return state_abbreviations[state_name]

In [63]:
# Adjust for area differences
land_df = pd.read_excel(os.path.join(current_dir, '..', 'Data', 'LandArea.xls'), dtype=str)
land_df = land_df[["STCOU", "LND110210D"]] # LND110210D is Land Area of counties in 2010 (not including water), STCOU is state fips and county fips
land_df["STATE_FIPS"] = land_df["STCOU"].str[:-3] # Extract just state fips
land_df["CNTY_FIPS"] = land_df["STCOU"].str[-3:] # Extract just county fips

counties = gpd.read_file(os.path.join(current_dir, '..', 'SharedData', 'UScounties', 'UScounties.shp')).drop("geometry", axis=1) # County dataset

merged1 = pd.merge(land_df, counties, on=["STATE_FIPS", "CNTY_FIPS"], how="inner", validate="1:1")
merged1['statecode'] = merged1['STATE_NAME'].apply(translate_state)
merged1 = merged1.rename(columns={"NAME": "COUNTY"})
grouped_area = grouped_area.rename(columns={"countyname": "COUNTY"})
merged_df = pd.merge(merged1, grouped_area, on=['statecode'], suffixes=('_df1', '_df2'))
def merge_county(df1_county, df2_county):
    return df2_county.startswith(df1_county)
merged_df = merged_df[merged_df.apply(lambda x: merge_county(x['COUNTY_df1'], x['COUNTY_df2']), axis=1)]

merged_df["bldg_area_per_sq_mile"] = merged_df["area_sum"] / merged_df["LND110210D"].astype(float)
merged_df.sort_values("bldg_area_per_sq_mile", ascending=False)

adjusted_grouped_area = merged_df[["STATE_NAME", "statecode", "STATE_FIPS", "COUNTY_df1", "COUNTY_df2", "CNTY_FIPS", "area_sum", "LND110210D", "bldg_area_per_sq_mile"]]
adjusted_grouped_area = adjusted_grouped_area.rename(columns={"LND110210D": "LandAreaSqMiles", "COUNTY_df1":"COUNTY", "statecode":"STATECODE", "area_sum":"total_bldg_area", "COUNTY_df2":"FORMAL_COUNTY"})
adjusted_grouped_area.head(10)

,STATE_NAME,STATECODE,STATE_FIPS,COUNTY,FORMAL_COUNTY,CNTY_FIPS,total_bldg_area,LandAreaSqMiles,bldg_area_per_sq_mile
0,Alabama,AL,01,Autauga,Autauga County,001,6993000,594.44,11764.013189
68,Alabama,AL,01,Baldwin,Baldwin County,003,60335200,1589.78,37951.917875
136,Alabama,AL,01,Barbour,Barbour County,005,6098300,884.88,6891.668927
204,Alabama,AL,01,Bibb,Bibb County,007,1386100,622.58,2226.380545
272,Alabama,AL,01,Blount,Blount County,009,2821200,644.78,4375.445889
340,Alabama,AL,01,Bullock,Bullock County,011,553000,622.81,887.911241
408,Alabama,AL,01,Butler,Butler County,013,2858700,776.83,3679.955717
476,Alabama,AL,01,Calhoun,Calhoun County,015,23999100,605.87,39610.972651
544,Alabama,AL,01,Chambers,Chambers County,017,6910000,596.53,11583.658827
612,Alabama,AL,01,Cherokee,Cherokee County,019,1849000,553.7,3339.353440


In [11]:
grouped.to_csv(os.path.join(current_dir, '..', 'SharedData', 'County_Building_Counts.csv'), index=False)
grouped_area.to_csv(os.path.join(current_dir, '..', 'SharedData', 'County_Building_Areas.csv'), index=False)
adjusted_grouped_area.to_csv(os.path.join(current_dir, '..', 'SharedData', 'County_Building_Areas_Adjusted.csv'), index=False)

In [56]:
grouped_use2 = grouped_use.pivot(columns="reported_propertytype", index=["statecode", "countyname"], values=["bldg_count", "area_sum"])
grouped_use2.columns = grouped_use2.columns.map('_'.join).str.strip('_')
grouped_use2 = grouped_use2.reset_index()
grouped_use2 = grouped_use2.fillna(0)
grouped_use2 = grouped_use2.drop(columns=["bldg_count_General Retail", "area_sum_General Retail", "bldg_count_Unknown", "area_sum_Unknown"])
grouped_use2 = grouped_use2.rename(columns={
    "bldg_count_Health Care": "bldg_count_Healthcare", 
    "area_sum_Health Care": "area_sum_Healthcare", 
    "bldg_count_Multi-Family": "bldg_count_Multifamily",
    "area_sum_Multi-Family": "area_sum_Multifamily",
    "bldg_count_Sports & Entertainment": "bldg_count_Entertainment",
    "area_sum_Sports & Entertainment": "area_sum_Entertainment"
    })
grouped_use2.to_csv(os.path.join(current_dir, '..', 'SharedData', 'County_Building_Counts_By_Type.csv'), index=False)
grouped_use2[grouped_use2["statecode"] == "MD"].head(10)

,statecode,countyname,bldg_count_Flex,bldg_count_Healthcare,bldg_count_Hospitality,bldg_count_Industrial,bldg_count_Multifamily,bldg_count_Office,bldg_count_Retail,bldg_count_Specialty,bldg_count_Entertainment,area_sum_Flex,area_sum_Healthcare,area_sum_Hospitality,area_sum_Industrial,area_sum_Multifamily,area_sum_Office,area_sum_Retail,area_sum_Specialty,area_sum_Entertainment
1193,MD,Allegany County,9.0,7.0,16.0,56.0,83.0,118.0,365.0,44.0,3.0,26400.0,854600.0,432200.0,3274900.0,1849400.0,1051600.0,4230400.0,408200.0,59500.0
1194,MD,Anne Arundel County,361.0,45.0,91.0,463.0,218.0,1173.0,2111.0,307.0,8.0,12761000.0,2227900.0,7290200.0,26285800.0,41517900.0,26397700.0,31959700.0,9129000.0,75700.0
1195,MD,Baltimore County,483.0,91.0,62.0,766.0,438.0,2046.0,3337.0,413.0,15.0,17502500.0,9903100.0,5578300.0,55640100.0,96946900.0,46348600.0,47629800.0,12397900.0,276900.0
1196,MD,Baltimore city,215.0,103.0,70.0,1435.0,2028.0,1880.0,5029.0,957.0,17.0,5986700.0,13221400.0,9457200.0,53948100.0,97233200.0,56664000.0,30040900.0,21070600.0,1717300.0
1197,MD,Calvert County,29.0,1.0,11.0,65.0,21.0,138.0,286.0,29.0,1.0,440900.0,441900.0,494600.0,1037000.0,971500.0,1326100.0,3182100.0,411200.0,1500.0
1198,MD,Caroline County,12.0,5.0,2.0,47.0,29.0,40.0,122.0,14.0,3.0,267000.0,134900.0,37300.0,2101400.0,1481400.0,257700.0,972200.0,285300.0,34900.0
1199,MD,Carroll County,75.0,22.0,6.0,203.0,117.0,293.0,626.0,67.0,4.0,1760900.0,1129900.0,182900.0,9635400.0,3819200.0,3161100.0,8545800.0,2354200.0,79400.0
1200,MD,Cecil County,16.0,8.0,16.0,121.0,90.0,142.0,449.0,44.0,3.0,442900.0,129000.0,522900.0,19595000.0,6910600.0,1140200.0,4220900.0,604600.0,95900.0
1201,MD,Charles County,48.0,8.0,20.0,174.0,68.0,252.0,632.0,89.0,2.0,776500.0,341600.0,712100.0,3245800.0,7036600.0,2758500.0,9138500.0,1738700.0,9000.0
1202,MD,Dorchester County,4.0,2.0,5.0,59.0,34.0,57.0,144.0,16.0,1.0,60600.0,62100.0,143100.0,2584200.0,1608300.0,495000.0,1418000.0,200000.0,6300.0


In [79]:
# Massive dataset
master = adjusted_grouped_area.merge(grouped, how="inner", right_on=["statecode", "countyname"], left_on=["STATECODE", "FORMAL_COUNTY"])
master = master.merge(grouped_use2, how="inner", on=["statecode", "countyname"])
master = master.drop(["STATECODE", "FORMAL_COUNTY"], axis=1)
cols = ["STATE_NAME", "statecode", "STATE_FIPS", "COUNTY", "countyname", "CNTY_FIPS", "bldg_count"] + master.columns.tolist()[4:7] + master.columns.tolist()[10:]
master = master[cols]
master = master.rename(columns={"statecode": "STATECODE", "countyname": "FORMAL_COUNTY"})
joined_fips_codes = master["STATE_FIPS"] + master["CNTY_FIPS"]
master.insert(6, column="FIPS", value=joined_fips_codes)
master.to_csv(os.path.join(current_dir, '..', 'SharedData', 'County_Buildings_MASTER.csv'), index=False)
master.head(10)

,STATE_NAME,STATECODE,STATE_FIPS,COUNTY,FORMAL_COUNTY,CNTY_FIPS,FIPS,bldg_count,total_bldg_area,LandAreaSqMiles,...,bldg_count_Entertainment,area_sum_Flex,area_sum_Healthcare,area_sum_Hospitality,area_sum_Industrial,area_sum_Multifamily,area_sum_Office,area_sum_Retail,area_sum_Specialty,area_sum_Entertainment
0,Alabama,AL,01,Autauga,Autauga County,001,01001,348,6993000,594.44,...,0.0,26900.0,9200.0,216300.0,2093700.0,1805400.0,453100.0,2223200.0,165200.0,0.0
1,Alabama,AL,01,Baldwin,Baldwin County,003,01003,3370,60335200,1589.78,...,13.0,724700.0,1414900.0,5791400.0,9663000.0,16491000.0,4850500.0,16964900.0,4044600.0,390200.0
2,Alabama,AL,01,Barbour,Barbour County,005,01005,183,6098300,884.88,...,2.0,0.0,0.0,64900.0,1786900.0,2974500.0,111500.0,1068400.0,86900.0,5200.0
3,Alabama,AL,01,Bibb,Bibb County,007,01007,97,1386100,622.58,...,0.0,19800.0,0.0,0.0,284200.0,437300.0,97100.0,526100.0,21600.0,0.0
4,Alabama,AL,01,Blount,Blount County,009,01009,266,2821200,644.78,...,3.0,31500.0,5400.0,0.0,794500.0,456800.0,152900.0,1221300.0,84300.0,74500.0
5,Alabama,AL,01,Bullock,Bullock County,011,01011,44,553000,622.81,...,0.0,0.0,0.0,0.0,229600.0,130300.0,47300.0,139500.0,6300.0,0.0
6,Alabama,AL,01,Butler,Butler County,013,01013,188,2858700,776.83,...,0.0,23000.0,85500.0,103800.0,894000.0,598200.0,89700.0,1018000.0,46500.0,0.0
7,Alabama,AL,01,Calhoun,Calhoun County,015,01015,1313,23999100,605.87,...,6.0,286300.0,241900.0,439000.0,6610000.0,4685100.0,2005600.0,8181100.0,1515000.0,35100.0
8,Alabama,AL,01,Chambers,Chambers County,017,01017,210,6910000,596.53,...,0.0,37300.0,32000.0,68300.0,3436100.0,1317700.0,603700.0,1251200.0,163700.0,0.0
9,Alabama,AL,01,Cherokee,Cherokee County,019,01019,136,1849000,553.7,...,0.0,5000.0,50000.0,54000.0,138700.0,394000.0,77500.0,1036500.0,93300.0,0.0
